In [ ]:

# data processing packages
import numpy as np   
import pandas as pd 

from scipy import stats # look at scipy
from scipy import linalg
from scipy import *

import random

# machine leanring packages

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing


from keras.models import Sequential 
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import LSTM
from keras.optimizers import *
from keras.callbacks import *
# RMSprop, Adadelta
from keras.regularizers import *
from keras.initializers import *
from keras.activations import *

from utils_keras import *
from utils_dataPrepro import *

# from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model
from keras.layers import *
from keras.layers.merge import *
from keras import backend as K


In [ ]:
# Notes:

# ---Ini:            
#     orthogonal: https://smerity.com/articles/2016/orthogonal_init.html
#     identity, Xavier, He's

# ---Activation:     
#     tan, sigmod by default

# ---Regularization: 
#     dropout on non-recurrent connections 
#     batch normalization

# unstable or not decreasing training loss: lr, representability, e.g., number of neursons, layers, etc..
# watch out for the amount difference between regularization and loss  
# large minibatch -> large lr
# large network -> large lr


# learning speed, repren, regular

# TO DO:
    
# more component
# muliti-input-muliti-output method
# log transformation
# integrate trend 
# attention for peridoic time series

# multistep ahead:  multi-output rnn, a.k.a. state space model 
#                   seq2seq rnn 

# Stat-space model: 


In [ ]:

files_list=["../../dataset/dataset_ts/air_xtrain.dat", \
            "../../dataset/dataset_ts/air_xtest.dat",\
            "../../dataset/dataset_ts/air_ytrain.dat", \
            "../../dataset/dataset_ts/air_ytest.dat"]

xtrain, ytrain, xtest, ytest, tr_shape, ts_shape = \
prepare_train_test_data( True, files_list)


xtrain = np.reshape( xtrain, (tr_shape[0], tr_shape[1]-1, -1) )
ytrain = np.reshape( ytrain, (tr_shape[0], 1) ) 
xtest  = np.reshape( xtest,  (ts_shape[0], ts_shape[1]-1, -1) )
ytest = np.reshape(  ytest,  (ts_shape[0], 1) )

print np.shape(xtrain), np.shape(ytrain), np.shape(xtest), np.shape(ytest)


In [ ]:
def split_raw_diff(x):
    cnt = len( list(x) )
    steps = len(x[0])
    
    raw  = []
    diff = []
    last = []
    
    for i in xrange(cnt):
        raw.append( [j[0] for j in x[i]] )
        last.append( x[i][steps-1][0] )
        diff.append( [j[1] for j in x[i]] )

    return np.asarray(raw), np.asarray(diff), np.asarray(last)

In [ ]:
raw_tr, diff_tr, last_tr = split_raw_diff(xtrain)
raw_ts, diff_ts, last_ts = split_raw_diff(xtest)

raw_tr  = np.reshape( raw_tr,  (tr_shape[0], tr_shape[1]-1, -1) )
diff_tr = np.reshape( diff_tr, (tr_shape[0], tr_shape[1]-1, -1) )
last_tr = np.reshape( last_tr, (tr_shape[0],  -1) )

raw_ts  = np.reshape( raw_ts,  (ts_shape[0], ts_shape[1]-1, -1) )
diff_ts = np.reshape( diff_ts, (ts_shape[0], ts_shape[1]-1, -1) )
last_ts = np.reshape( last_ts, (ts_shape[0],  -1) )

ytrain = np.reshape( ytrain, (tr_shape[0], 1) ) 
ytest = np.reshape(  ytest,  (ts_shape[0], 1) )

print np.shape(raw_tr), np.shape(diff_tr), np.shape(last_tr)
print np.shape(raw_ts), np.shape(diff_ts), np.shape(last_ts)

In [ ]:
# plain discriminative 

# network set-up
in_out_neurons = 1
hidden_neurons = 256
win_size = 200

batch_size = 256

disc_xtrain = np.reshape( xtrain, [-1, win_size, in_out_neurons] )
disc_xtest  = np.reshape( xtest,  [-1, win_size, in_out_neurons] )

disc_ytrain = ytrain
disc_ytest  = ytest

print np.shape(disc_xtrain),np.shape(disc_ytrain),np.shape(disc_xtest),\
np.shape(disc_ytest)

# optimizer
sgd  = SGD(lr = 0.01, momentum = 0.9, nesterov = True)
rms  = RMSprop(lr = 0.05,  rho = 0.9, epsilon  = 1e-08, decay = 0.0)

adam = Adam(lr = 0.005, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-08, decay = 0.0)

model = Sequential()
model.add( LSTM(hidden_neurons, input_dim = in_out_neurons, return_sequences = False, \
                input_shape = [batch_size, win_size, in_out_neurons ], \
#               input_length = win_size, \
#               activation ='tanh',\
#               dropout = 0.1,\
#               kernel_regularizer = l2(0.2), 
#               recurrent_regularizer = l2(0.1),\
#               !change!
                kernel_initializer    = glorot_normal(), \
                recurrent_initializer = glorot_normal() ))
# change: activiation

# model.add( LSTM(hidden_neurons, return_sequences = False, \
# #               input_shape = [batch_size, win_size, in_out_neurons ], \
# #               input_length = win_size, \
#               activation ='tanh',\
# #               dropout = 0.1,\
# #               kernel_regularizer = l2(0.2), 
# #               recurrent_regularizer = l2(0.1),\
# #               !change!
#                 kernel_initializer    = glorot_normal(), \
#                 recurrent_initializer = glorot_normal() ))


# identity ini
# he's ini:   he_normal()
# xavier ini: glorot_normal()
# orthogonal: Orthogonal()


# model.add(Dropout(0.5))
model.add(Dense(128,\
#                   kernel_regularizer = l2(0.01), \
                    kernel_initializer = he_normal()))
model.add(Activation("relu"))
model.add(Dense(64, \
#                   kernel_regularizer = l2(0.01), \
                    kernel_initializer = he_normal()))
model.add(Activation("relu"))
model.add(Dense(32, \
#                   kernel_regularizer = l2(0.01), \
                    kernel_initializer = he_normal()))
model.add(Activation("relu"))
model.add(Dense(1,  \
#                   kernel_regularizer = l2(0.01), \
                    kernel_initializer = he_normal()))
model.add(Activation("linear"))


# training
model.compile(loss = "mean_squared_error", optimizer = adam)


model.fit( disc_xtrain, disc_ytrain, shuffle = True, \
           callbacks = [ \
           TestCallback( (disc_xtest, disc_ytest), (disc_xtrain, disc_ytrain) ) ], \
           batch_size = batch_size, epochs = 500)


In [ ]:
# Generative model

# set-up
in_out_neurons = 1
hidden_neurons = 256
win_size = 200    
    
# prepare data
ytrain = expand_y( xtrain_df.as_matrix(), ytrain_df.as_matrix() )
ytest  = expand_y( xtest_df.as_matrix(),  ytest_df.as_matrix()  )

gen_xtrain = np.reshape( xtrain, [-1, win_size, 1] )
gen_ytrain = np.reshape( ytrain, [-1, win_size, 1] )

gen_xtest  = np.reshape( xtest, [-1, win_size, 1] )
gen_ytest  = np.reshape( ytest, [-1, win_size, 1] )

print np.shape(gen_xtrain), np.shape(gen_ytrain), np.shape(gen_xtest), np.shape(gen_ytest)


# optimizer 
adam  = Adam(lr = 0.002, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-08, decay = 0.0)

model = Sequential()
model.add( LSTM(hidden_neurons, input_dim = in_out_neurons, return_sequences = True, \
                input_length = win_size,\
                activation   = 'tanh',\
#                dropout = 0.1,\
#                kernel_regularizer = l2(0.15), \
#                recurrent_regularizer = l2(0.15), \
                kernel_initializer    = glorot_normal(), \
                recurrent_initializer = glorot_normal() ) )


# identity ini
# he's ini:   he_normal()
# xavier ini: glorot_normal()
# orthogonal: Orthogonal() 


# model.add(Dropout(0.1))
model.add(TimeDistributed(Dense(128, activation = 'relu',\
#                                     kernel_regularizer = l2(0.1), \
                                    kernel_initializer = he_normal() ) ))

model.add(TimeDistributed(Dense(64, activation = 'relu',\
#                                     kernel_regularizer = l2(0.01), \
                                    kernel_initializer = he_normal() ) ))

model.add(TimeDistributed(Dense(1,  activation = 'linear',\
#                                     kernel_regularizer = l2(0.01), \
                                    kernel_initializer = he_normal() ) ))


# training
model.compile( loss = "mean_squared_error", optimizer = adam )


model.fit( gen_xtrain, gen_ytrain, shuffle=True,  \
           callbacks = [ TestCallback_Generative( \
                         (gen_xtest, gen_ytest), (gen_xtrain, gen_ytrain) ) ], \
                         batch_size = 128, epochs = 500 )

In [ ]:
# State space model 
# with non-overlapping local data

#  network set-up
local_size = 5
in_out_neurons = local_size
hidden_neurons = 512
win_size = 200/5

# # validation on each epoch 
# class TestCallback_insight(Callback):
#     def __init__(self, test_data):
#         self.test_data = test_data

#     def on_epoch_end(self, epoch, logs={}):
#         x, y = self.test_data
#         loss, mse = self.model.evaluate(x, y, verbose=0)
        
#         py = self.model.predict(x, verbose=0)
#         cnt = len(y)
#         err = [ (y[i][0]-py[i][0])**2 for i in range(cnt)]
        
#         print('\nTesting loss: {}, acc: {}, mean: {} \n'.format(\
#                loss, sqrt(mse), mean(err) ))

def expand_x_local_non_overlapping( local_size, list_data):
    
    cnt = len(list_data)
    steps = len(list_data[0])
    tmp_dta = []
    
    for i in range(cnt):
        tmp_dta.append([])
        for j in range( local_size-1, steps, local_size ):
            tmp_dta[-1].append( list_data[i][ j-local_size+1:j+1 ] )
    
    return tmp_dta


disc_xtrain = np.array( expand_x_local_non_overlapping( local_size, list(xtrain) ) )
disc_xtest  = np.array( expand_x_local_non_overlapping( local_size, list(xtest) ) )

disc_ytrain = ytrain
disc_ytest  = ytest

print np.shape(disc_xtrain), np.shape(disc_ytrain), np.shape(disc_xtest), \
np.shape(disc_ytest)

# optimizer 
adam = Adam(lr = 0.007, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-08, decay = 0.0)

model = Sequential()
model.add(LSTM(hidden_neurons, input_dim = in_out_neurons, return_sequences = False, \
               input_length = win_size, \
               activation = 'tanh', \
#                dropout = 0.1,\
#                kernel_regularizer = l2(0.1),      recurrent_regularizer = l2(0.1), \
               kernel_initializer    = glorot_normal(), \
               recurrent_initializer = glorot_normal() ))
# identity ini
# he's ini:   he_normal()
# xavier ini: glorot_normal()
# orthogonal: Orthogonal() 

model.add(Dropout(0.1))

# kernel_regularizer = l2(0.1), \
model.add(Dense(128,\
                    kernel_initializer = he_normal()))
model.add(Activation("relu"))
model.add(Dense(64, \
                    kernel_initializer = he_normal()))
model.add(Activation("relu"))
model.add(Dense(32, \
                    kernel_initializer = he_normal()))
model.add(Activation("relu"))
model.add(Dense(1,\
                    kernel_initializer = he_normal()))
model.add(Activation("linear"))


# training 
model.compile(loss="mean_squared_error", optimizer = adam)


model.fit( disc_xtrain, disc_ytrain, shuffle=True, \
           callbacks = [ TestCallback( (disc_xtest, disc_ytest), (disc_xtrain, disc_ytrain) ) ], \
           batch_size = 256, epochs = 500)

In [ ]:
# validation on each epoch 
class TestCallback_ada(Callback):
    def __init__(self, test_data, train_data):
        
        self.test_data  = test_data        
        self.train_data = train_data

    def on_epoch_end(self, epoch, logs={}):
        
        raw_ts, diff_ts, last_ts, y_ts = self.test_data
        raw_tr, diff_tr, last_tr, y_tr = self.train_data
        
        py_tr = self.model.predict( [raw_tr, diff_tr, last_tr], verbose=0)
        py_ts = self.model.predict( [raw_ts, diff_ts, last_ts], verbose=0)
        
        size_tr = len(y_tr)
        size_ts = len(y_ts)
        
        err_tr = [ (y_tr[i][0] - py_tr[i][0])**2 for i in range(size_tr) ]
        err_ts = [ (y_ts[i][0] - py_ts[i][0])**2 for i in range(size_ts) ]
        
        loss = self.model.evaluate([raw_ts, diff_ts, last_ts], y_ts, verbose=0)
        
        with open("res/tsRnn_ada.txt", "a") as text_file:
            text_file.write("At epoch %d: loss %f, train %f, test %f\n" % ( epoch, loss, sqrt(mean(err_tr)),\
                                                                           sqrt(mean(err_ts))) )

In [ ]:

#--- PARAMETERS ---
N_DIM = 1
N_RNN = 64

LR = 0.005

WIN_SIZE = 200

MODEL_CHECK_PERIOD = 2
BATCH_SIZE = 256

# --- Prepare ----

input_raw  = Input(shape = ( WIN_SIZE -1, 1 ), dtype='float32', name='input_raw')
input_diff = Input(shape = ( WIN_SIZE -1, 1 ), dtype='float32', name='input_diff')
# smoothing last values
input_last = Input(shape = ( 1, ), dtype='float32', name='input_last')

with open("res/tsRnn_ada.txt", "w") as text_file:
    text_file.close()
        
# --- build the network ---    

def difference(pair_of_tensors):
    x, y = pair_of_tensors
    return x - y

def one_minus(tensor):
    
    return K.ones(1) - tensor
    
#     return [1.0-i for i in tensor]

hidden_raw  = LSTM(N_RNN)( input_raw )
output_raw  = Dense(1, activation='relu')(hidden_raw)

hidden_diff = LSTM(N_RNN)( input_diff )
output_diff = Dense(1, activation='relu')(hidden_diff)
output_diff = Add()( [output_diff, input_last] )

weight_var = Lambda(difference)([output_diff, output_raw])

diff_prob = Dense(1, activation='sigmoid')( weight_var )
weighted_diff = merge([output_diff, diff_prob], name='weighted_diff', mode='mul')

raw_prob = Lambda(one_minus)([diff_prob])
weighted_raw  = merge([output_raw, raw_prob], name='attention_raw', mode='mul')

output_main = Add()( [weighted_diff, weighted_raw] )


# --- training ---

model = Model(inputs=[ input_raw, input_diff, input_last], outputs=[ output_main ])

adam = Adam(lr = LR , beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-08, decay = 0.0)

model.compile(optimizer = adam, loss='mean_squared_error')

filepath="res/model/weights-{epoch:02d}.hdf5"
checkpointer = ModelCheckpoint(filepath, verbose=0, save_best_only=False, save_weights_only=False,\
                               period=MODEL_CHECK_PERIOD)

model.fit([raw_tr, diff_tr, last_tr], [ytrain], epochs = 500, batch_size = BATCH_SIZE,\
          callbacks = \
          [TestCallback_ada( [ raw_tr, diff_tr, last_tr, ytrain ], \
                             [ raw_ts, diff_ts, last_ts, ytest ] ), checkpointer ])
                  

# # load weights
# model.load_weights("weights.best.hdf5")
# # Compile model (required to make predictions)
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
